In [1]:
from datasets import load_happines
from tools import *
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import seaborn as sns
import plotly.express as px
import pandas as pd
from sklearn.model_selection import cross_val_score

In [2]:
df_train, df_test = load_happines()
df_train.columns

Index(['region', 'economy', 'family', 'health', 'freedom', 'trust',
       'generosity', 'target'],
      dtype='object')

In [4]:
train_X = df_train.drop(['region', 'target'], axis=1)
train_y = df_train['target']
test_X = df_test.drop(['region', 'target'], axis=1)
test_y = df_test['target']

In [5]:
pca = PCA(n_components=3)
pca = pca.fit(train_X)

In [6]:
pca.explained_variance_ratio_

array([0.71380549, 0.11285944, 0.07646787])

In [7]:
pca.singular_values_

array([6.27096091, 2.49352139, 2.05250222])

In [8]:
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [9]:
transformed_df = pd.DataFrame(transformed_train_X, columns=['x','y','z'])
transformed_df['target'] = train_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target', size='target')

In [10]:
transformed_df = pd.DataFrame(transformed_test_X, columns=['x','y','z'])
transformed_df['target'] = test_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target', size='target')

In [12]:
cross_val_score(LinearRegression(), transformed_train_X, train_y).mean()

0.7531151819481541

In [13]:
model = LinearRegression()
model = model.fit(transformed_train_X, train_y)

In [14]:
model_performance(model, transformed_train_X, train_y, transformed_test_X, test_y)

Train score
0.7753601411472273
Test score
0.7115720517177949


In [15]:
coef_table(model.coef_, ['x', 'y', 'z'])

,feature,coef
2,z,0.998199
1,y,-1.102530
0,x,-1.938931


In [17]:
scores = []
components = []
for i in range(1, len(df_train.columns) - 1):
    pca = PCA(n_components=i)
    pca = pca.fit(train_X)
    transformed_train_X = pca.transform(train_X)
    transformed_test_X = pca.transform(test_X)
    components.append(i)
    scores.append(cross_val_score(LinearRegression(), transformed_train_X, train_y).mean())

pd.DataFrame(zip(components, scores), columns=['components', 'score'])

,components,score
0,1,0.704326
1,2,0.740667
2,3,0.753115
3,4,0.749286
4,5,0.747060
5,6,0.742787
